# Production Error Handling Utilities

Reusable custom exceptions and context managers for real applications.

## 1. Application Exception Hierarchy

In [ ]:
"""Base exception hierarchy for applications."""

class ApplicationError(Exception):
    """Base exception for all application errors."""
    def __init__(self, message: str, code: str | None = None):
        super().__init__(message)
        self.message = message
        self.code = code or self.__class__.__name__
    
    def to_dict(self) -> dict:
        return {
            "error": self.code,
            "message": self.message,
        }


class ValidationError(ApplicationError):
    """Input validation failed."""
    def __init__(self, message: str, field: str | None = None):
        super().__init__(message, code="VALIDATION_ERROR")
        self.field = field
    
    def to_dict(self) -> dict:
        result = super().to_dict()
        if self.field:
            result["field"] = self.field
        return result


class NotFoundError(ApplicationError):
    """Resource not found."""
    def __init__(self, resource: str, identifier: str | int):
        super().__init__(f"{resource} '{identifier}' not found", code="NOT_FOUND")
        self.resource = resource
        self.identifier = identifier


class AuthenticationError(ApplicationError):
    """Authentication failed."""
    def __init__(self, message: str = "Authentication required"):
        super().__init__(message, code="AUTH_ERROR")


class AuthorizationError(ApplicationError):
    """User lacks permission."""
    def __init__(self, action: str, resource: str | None = None):
        msg = f"Not authorized to {action}"
        if resource:
            msg += f" on {resource}"
        super().__init__(msg, code="FORBIDDEN")


class RateLimitError(ApplicationError):
    """Rate limit exceeded."""
    def __init__(self, retry_after: int):
        super().__init__(f"Rate limit exceeded. Retry after {retry_after}s", code="RATE_LIMITED")
        self.retry_after = retry_after


# Test
try:
    raise NotFoundError("User", 123)
except ApplicationError as e:
    print(e.to_dict())

## 2. API/External Service Errors

In [ ]:
"""Exceptions for external API calls."""

class ExternalServiceError(ApplicationError):
    """Base for external service errors."""
    def __init__(self, service: str, message: str):
        super().__init__(f"{service}: {message}", code="EXTERNAL_ERROR")
        self.service = service


class APIError(ExternalServiceError):
    """HTTP API error."""
    def __init__(
        self, 
        service: str, 
        status_code: int, 
        message: str,
        response_body: dict | None = None
    ):
        super().__init__(service, f"HTTP {status_code}: {message}")
        self.status_code = status_code
        self.response_body = response_body or {}


class LLMError(ExternalServiceError):
    """LLM API error."""
    def __init__(self, provider: str, message: str, model: str | None = None):
        super().__init__(f"LLM/{provider}", message)
        self.provider = provider
        self.model = model


class LLMRateLimitError(LLMError):
    """LLM rate limit hit."""
    def __init__(self, provider: str, retry_after: float | None = None):
        super().__init__(provider, "Rate limit exceeded")
        self.retry_after = retry_after


class LLMContextLengthError(LLMError):
    """Context too long for model."""
    def __init__(self, provider: str, model: str, max_tokens: int, actual_tokens: int):
        super().__init__(provider, f"Context length {actual_tokens} exceeds max {max_tokens}", model)
        self.max_tokens = max_tokens
        self.actual_tokens = actual_tokens


# Test
try:
    raise LLMContextLengthError("OpenAI", "gpt-4", 8192, 10000)
except LLMError as e:
    print(f"Error: {e}")
    print(f"Provider: {e.provider}")

## 3. Context Managers

In [ ]:
"""Reusable context managers."""

import time
from contextlib import contextmanager
from typing import Generator


@contextmanager
def timer(name: str = "Block") -> Generator[None, None, None]:
    """Time a code block and print duration."""
    start = time.perf_counter()
    try:
        yield
    finally:
        elapsed = time.perf_counter() - start
        print(f"{name} took {elapsed:.4f}s")


@contextmanager
def suppress_and_log(*exceptions, logger=None):
    """Suppress exceptions but log them."""
    try:
        yield
    except exceptions as e:
        msg = f"Suppressed {type(e).__name__}: {e}"
        if logger:
            logger.warning(msg)
        else:
            print(msg)


# Test timer
with timer("Sleep test"):
    time.sleep(0.3)

# Test suppress
with suppress_and_log(ValueError, KeyError):
    raise ValueError("This is suppressed")

print("Continued after suppressed error")

In [ ]:
"""File and resource context managers."""

import tempfile
import shutil
from pathlib import Path
from contextlib import contextmanager


@contextmanager
def temp_directory():
    """Create temp directory, auto-cleanup after."""
    path = Path(tempfile.mkdtemp())
    try:
        yield path
    finally:
        shutil.rmtree(path, ignore_errors=True)


@contextmanager
def atomic_write(filepath: str | Path, mode: str = "w"):
    """Write to file atomically (write to temp, then rename)."""
    filepath = Path(filepath)
    temp_path = filepath.with_suffix(filepath.suffix + ".tmp")
    try:
        with open(temp_path, mode) as f:
            yield f
        temp_path.rename(filepath)
    except:
        temp_path.unlink(missing_ok=True)
        raise


# Test temp_directory
with temp_directory() as tmp:
    print(f"Temp dir: {tmp}")
    (tmp / "test.txt").write_text("hello")

print(f"Cleaned up: {not tmp.exists()}")

## All-in-One: exceptions.py

In [ ]:
# exceptions.py - Copy to create module
"""
Application exception hierarchy and context managers.

Usage:
    from exceptions import ApplicationError, NotFoundError, timer
"""

__all__ = [
    # Base
    "ApplicationError",
    "ValidationError", 
    "NotFoundError",
    "AuthenticationError",
    "AuthorizationError",
    "RateLimitError",
    # External
    "ExternalServiceError",
    "APIError",
    "LLMError",
    "LLMRateLimitError",
    "LLMContextLengthError",
    # Context Managers
    "timer",
    "suppress_and_log",
    "temp_directory",
    "atomic_write",
]